In [27]:
import os
import re
import cv2 # opencv library
import numpy as np
from os.path import isfile, join
import matplotlib.pyplot as plt
min_cont_width=50 
min_cont_height=50
offset=10
line_pos=480 
detected =[]
vehicles=0
def mid_points(x, y, w, h):

    cx = int((x + (x+w))/2)
    cy = int((y + (y+h))/2)
    return cx,cy
 
cap = cv2.VideoCapture('Night Time Traffic Camera video.mp4')

if cap.isOpened():
    ret,frame1 = cap.read()
else:
    ret = False
ret,frame1 = cap.read()
ret,frame2 = cap.read()
    
while ret:
    d = cv2.absdiff(frame1,frame2)
    grey = cv2.cvtColor(d,cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(grey,(5,5),0)

    ret , th = cv2.threshold(blur,20,255,cv2.THRESH_BINARY)
    
    dilated = cv2.dilate(th,np.ones((3,3)))
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))

    closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel) 
    
    contours,h = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        (x,y,w,h) = cv2.boundingRect(c)
        contour_valid = (w > min_cont_width) and  (h > min_cont_height) 
        
        if not contour_valid:
            continue
        cv2.line(frame1, (0, line_pos), (1200, line_pos), (0,255,0), 2)
        cv2.line(frame1, (0,line_pos-offset), (1200,line_pos-offset), (0,0,255), 1) # Offset ABOVE
        cv2.line(frame1, (0,line_pos+offset), (1200,line_pos+offset), (0,0,255), 1) # Offset BELOW
        
        if x > 10 and x < 1300 and y > 100:
            cv2.rectangle(frame1,(x-10,y-10),(x+w+10,y+h+10),(255,0,0),2)
            centroid = mid_points(x, y, w, h)
            detected.append(centroid)
        
            cv2.circle(frame1,centroid, 5, (0,0,255), 5)
            cx,cy= mid_points(x, y, w, h)
            
            for (x,y) in detected:
                if y<(line_pos+offset) and y>(line_pos-offset) :
                    vehicles=vehicles+1
                    detected.remove((x,y))
                    print(vehicles)
                
    cv2.putText(frame1, "Total vehicles Detected: " + str(vehicles), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 0, 255), 2)


    cv2.imshow("Original" , frame1)
    cv2.imshow("Difference" , th)
    if cv2.waitKey(1) == 27:
        break
    frame1 = frame2
    ret , frame2 = cap.read()
#print(matches)    
cv2.destroyAllWindows()
cap.release()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
